In [12]:
%config Completer.use_jedi = False  #fixing an autocomplete bug in my jupyter, you might not want this
import sympy as sp 
import numpy as np
from mpmath import *
from sympy import *  #oops lazy, sorry

### I define so many symbols to have covariant and contravariant versions of the necessary vectors for at least two different indices. $\rho$ is used as an index for cases where vectors are multiplied over the same index, e.g., $x^\rho z_\rho$
#### 
#### I will just use Bordovitsyn's 2003 paper
###### https://link.springer.com/content/pdf/10.1023/A:1026217822337.pdf
#### and Jackson and my thesis.
#### 
#### There are many useful identities in Bordovitsyn's paper, particularly the bottom equation from page 460: $\tilde{r}_{\rho} v^{\rho}=-\tilde{r} c \gamma[1-(\tilde{n} \boldsymbol{\beta})]$. Equation 26 and the other equations on that page are very helpful for understanding the necessary parts for doing covariant derivatives on the LW potentials.

In [13]:
Pcv_aph = sp.symbols("P^alpha",real=True)
Pcn_aph = sp.symbols("P_alpha",real=True)
Acv_aph = sp.symbols("A^alpha",real=True)
Acn_aph = sp.symbols("A_alpha",real=True)
vcv_aph = sp.symbols("v^alpha",real=True)
vcn_aph = sp.symbols("v_alpha",real=True)
rcn_aph = sp.symbols(r"\tilde{r}_{\alpha}",real=True)
rcv_aph = sp.symbols(r"\tilde{r}^{\alpha}",real=True)

Pcv_bt= sp.symbols("P^beta",real=True)
Pcn_bt = sp.symbols("P_beta",real=True)
Acv_bt = sp.symbols("A^beta",real=True)
Acn_bt = sp.symbols("A_beta",real=True)
vcv_bt = sp.symbols("v^beta",real=True)
vcn_bt = sp.symbols("v_beta",real=True)
rcn_bt = sp.symbols(r"\tilde{r}_{\beta}",real=True)
rcv_bt = sp.symbols(r"\tilde{r}^{\beta}",real=True)

r_x = sp.symbols(r"\tilde{r}^x",real=True,positive=True)
r_y = sp.symbols(r"\tilde{r}^y",real=True,positive=True)
r_z = sp.symbols(r"\tilde{r}^z",real=True,positive=True)

v_x = sp.symbols(r"\tilde{v}^x",real=True,positive=True)
v_y = sp.symbols(r"\tilde{v}^y",real=True,positive=True)
v_z = sp.symbols(r"\tilde{v}^z",real=True,positive=True)

rcn_rho = sp.symbols(r"\tilde{r}_{\rho}",real=True)
rcv_rho = sp.symbols(r"\tilde{r}^{\rho}",real=True)
vcn_rho = sp.symbols("v_rho",real=True)
vcv_rho = sp.symbols("v^rho",real=True)
gup_ab =  sp.symbols("g^alpha^beta",real=True)
glw_ab =  sp.symbols("g_alpha_beta",real=True)
gup_ba =  sp.symbols("g^beta^alpha",real=True)
glw_ba = sp.symbols("g_beta_alpha",real=True)
del_ab = sp.symbols("\delta^alpha_beta",real=True)

q =  sp.symbols("q",real=True)
q_s =  sp.symbols("q_s",real=True)
c =  sp.symbols("c",real=True,positive=True)
K = sp.symbols("K",real=True,positive=True)

gm = sp.symbols("gamma",real=True,positive=True)
bta = sp.symbols(r'\vec{\beta}',real=True)
dbta = sp.symbols(r'\dot{\vec{\beta}}',real=True)
bta_dot_dbta = sp.symbols(r'\vec{\beta}\cdot\dot{\vec{\beta}}',real=True)
bta_ob_dot_dbta = sp.symbols(r'\beta_{ob}\cdot\dot{\vec{\beta}}',real=True)

bta_x = sp.symbols(r'\beta_x',real=True,positive=True)
bta_x_ob = sp.symbols(r'\beta_{x-ob}',real=True,positive=True)

#dvdtau = sp.symbols(r'\frac{dv^\beta}{d\tau}',real=True) 
dvdtau_bt = sp.symbols(r'\dot{V^\beta}',real=True) 
dvdtau_aph = sp.symbols(r'\dot{V^\alpha}',real=True) 

tau = sp.symbols('tau',real=True) 
r_mag = sp.symbols(r"|\tilde{r}|",real=True)  #same as jackson's |R|
n_src_ob = sp.symbols(r"|\hat{n}|",real=True)  #same as jackson's |R|
n_src_ob_x = sp.symbols(r"|\hat{n}|_x",real=True)  #same as jackson's |R|
n_src_ob_y = sp.symbols(r"|\hat{n}|_y",real=True)  #same as jackson's |R|
n_src_ob_z = sp.symbols(r"|\hat{n}|_z",real=True)  #same as jackson's |R|

#ONLY FOR OBJECTIVE
s_vec = sp.symbols(r'\vec{s}',real=True) #spin vector
m_ob = sp.symbols('m_{ob}',real=True,positive=True) 
gm_ob = sp.symbols(r'\gamma_{ob}',real=True) 
bta_ob = sp.symbols("beta_ob",real=True,positive=True)
vcn_bt_ob = sp.symbols("v_beta_-ob",real=True)
vcv_bt_ob = sp.symbols("v^beta^-ob",real=True)
vcv_aph_ob = sp.symbols("v^alpha^-ob",real=True)
vcn_aph_ob = sp.symbols("v_alpha_-ob",real=True)

#dot product
bta_dot_btaob =sp.symbols(r'\vec{\beta}\cdot\beta_{ob}',real=True,positive=True) 

#d/dtau derivatives (proper time)
bta_dot =sp.symbols(r'\dot{\beta}',real=True,positive=True)  
bta_dot_ob =sp.symbols(r'\dot{\beta}_{ob}',real=True,positive=True)  
bta_dot_x = sp.symbols(r'\dot{\beta}_{x}',real=True,positive=True)  


In [14]:
bta_dot_btaob

\vec{\beta}\cdot\beta_{ob}

### this block is just a test based on jackson's dVdtau formalism. i use bordovitsyn instead below 

In [15]:
#inferring from 14.11 a different form for dcv_aph_Acv_bt
#d_dtau_term = rcv_aph*vcv_bt/(K*c*r_mag*gm) #for reference only

d_dtau_term_solved = ((-vcv_aph*vcv_bt+rcv_aph*dvdtau_bt)*(K*c*r_mag*gm) - rcv_aph*vcv_bt*(-c**2+rcn_aph*dvdtau_aph))/(K*c*r_mag*gm)**2
dcv_aph_Acv_bt_alt = expand(q_s/(K*c*r_mag*gm)*d_dtau_term_solved)
#dcv_aph_Acv_bt_alt = dcv_aph_Acv_bt_alt.subs(rcn_aph*rcv_aph,0) #not wrong!!!!!
#dcv_aph_Acv_bt_alt = dcv_aph_Acv_bt_alt.subs(vcv_aph*vcv_bt,c**2*gup_ab)
#dcv_aph_Acv_bt_alt = dcv_aph_Acv_bt_alt.subs(rcn_bt*vcv_bt,(K*c*r_mag*gm))
#dcv_aph_Acv_bt_alt = dcv_aph_Acv_bt_alt.subs(rcv_aph*vcv_bt,(K*c*r_mag*gm))


#dcv_aph_Acv_bt_alt = dcv_aph_Acv_bt_alt.subs(rcv_aph*vcv_bt,gup_ab*(K*c*r_mag*gm))
dcv_aph_Acv_bt_alt

### when switching dVdtau indices, Jac(K*c*r_mag*gm)son is ambiguous here. Leading terms match bordovitsyn with switched indices, but loo(K*c*r_mag*gm)s suspect 

\dot{V^\beta}*\tilde{r}^{\alpha}*q_s/(K**2*c**2*gamma**2*|\tilde{r}|**2) - q_s*v^alpha*v^beta/(K**2*c**2*gamma**2*|\tilde{r}|**2) - \dot{V^\alpha}*\tilde{r}^{\alpha}*\tilde{r}_{\alpha}*q_s*v^beta/(K**3*c**3*gamma**3*|\tilde{r}|**3) + \tilde{r}^{\alpha}*q_s*v^beta/(K**3*c*gamma**3*|\tilde{r}|**3)

In [16]:
dcv_aph_Acv_bt_alt

\dot{V^\beta}*\tilde{r}^{\alpha}*q_s/(K**2*c**2*gamma**2*|\tilde{r}|**2) - q_s*v^alpha*v^beta/(K**2*c**2*gamma**2*|\tilde{r}|**2) - \dot{V^\alpha}*\tilde{r}^{\alpha}*\tilde{r}_{\alpha}*q_s*v^beta/(K**3*c**3*gamma**3*|\tilde{r}|**3) + \tilde{r}^{\alpha}*q_s*v^beta/(K**3*c*gamma**3*|\tilde{r}|**3)

In [17]:
d_dtau_term_solved_reverse = ((-vcv_aph*vcv_bt+rcv_bt*dvdtau_aph)*(K*c*r_mag*gm) - rcv_bt*vcv_aph*(-c**2+rcn_aph*dvdtau_bt))/(K*c*r_mag*gm)**2
dcv_bt_Acv_aph_alt = expand(q_s/(K*c*r_mag*gm)*d_dtau_term_solved_reverse)

In [18]:
expand(dcv_aph_Acv_bt_alt-dcv_bt_Acv_aph_alt)

-\dot{V^\alpha}*\tilde{r}^{\beta}*q_s/(K**2*c**2*gamma**2*|\tilde{r}|**2) + \dot{V^\beta}*\tilde{r}^{\alpha}*q_s/(K**2*c**2*gamma**2*|\tilde{r}|**2) - \dot{V^\alpha}*\tilde{r}^{\alpha}*\tilde{r}_{\alpha}*q_s*v^beta/(K**3*c**3*gamma**3*|\tilde{r}|**3) + \dot{V^\beta}*\tilde{r}^{\beta}*\tilde{r}_{\alpha}*q_s*v^alpha/(K**3*c**3*gamma**3*|\tilde{r}|**3) + \tilde{r}^{\alpha}*q_s*v^beta/(K**3*c*gamma**3*|\tilde{r}|**3) - \tilde{r}^{\beta}*q_s*v^alpha/(K**3*c*gamma**3*|\tilde{r}|**3)

In [19]:
d_dtau_term_solved#.subs(dvdtau_bt,0)

(K*c*gamma*|\tilde{r}|*(\dot{V^\beta}*\tilde{r}^{\alpha} - v^alpha*v^beta) - \tilde{r}^{\alpha}*v^beta*(\dot{V^\alpha}*\tilde{r}_{\alpha} - c**2))/(K**2*c**2*gamma**2*|\tilde{r}|**2)

In [20]:
#expand(dAdtau)

#### I use $\tilde{r}$ (Bordovitsyn's notation) for the distance between particles at retarded time $\tau$. This is $[x-r(\tau)]$ in Jackson, where  $\left|[x-r(\tau)]\right| \equiv R$ 

#### Starting with the second form of Equation 43 from Bordovitsyn's paper.
##### and using K $\equiv (1-\beta \cdot \vec{n})$

In [21]:
dcv_aph_Acv_bt = -vcv_aph/(c*(K)*r_mag*gm)*Acv_bt+rcv_aph/(c*K*r_mag*gm)*sp.Derivative(Acv_bt,tau)
dcv_aph_Acv_bt

-A^beta*v^alpha/(K*c*gamma*|\tilde{r}|) + \tilde{r}^{\alpha}*Derivative(A^beta, tau)/(K*c*gamma*|\tilde{r}|)

### Now we need Jackson 14.12, and 14.6



In [22]:
vcv_bt

v^beta

In [23]:
#using quotient rule... keep a close eye on indices here,
#dAdtau = expand(q_s*(dvdtau_bt*(K)*c*r_mag*gm-vcv_bt*(-c**2+rcn_bt*dvdtau_bt))/(K*c*r_mag*gm)**2)
dAdtau = expand(q_s*(dvdtau_bt*(K)*c*r_mag*gm-vcv_bt*(-c**2+rcn_aph*dvdtau_aph))/(K*c*r_mag*gm)**2)
#dAdtau = dAdtau.subs(rcn_bt*vcv_bt,K*c*r_mag*gm)
dAdtau

\dot{V^\beta}*q_s/(K*c*gamma*|\tilde{r}|) - \dot{V^\alpha}*\tilde{r}_{\alpha}*q_s*v^beta/(K**2*c**2*gamma**2*|\tilde{r}|**2) + q_s*v^beta/(K**2*gamma**2*|\tilde{r}|**2)

In [24]:
#dcv_aph_Acv_bt_sbbd = dcv_aph_Acv_bt.subs(sp.Derivative(Acv_bt,tau),expand(dAdtau))       #bordovitsyn
dcv_aph_Acv_bt_sbbd = dcv_aph_Acv_bt_alt#.subs(sp.Derivative(Acv_bt,tau),expand(dAdtau)) #mine by jackson
#dcv_aph_Acv_bt_sbbd = (expand(dcv_aph_Acv_bt_sbbd)) #original
#dcv_aph_Acv_bt_sbbd = (expand(dcv_aph_Acv_bt_sbbd)*rcn_aph) #extra simple
dcv_aph_Acv_bt_sbbd= dcv_aph_Acv_bt_sbbd.subs(Acv_bt,q_s*vcv_bt/(c*(K)*r_mag*gm))
#dcv_aph_Acv_bt_sbbd= dcv_aph_Acv_bt_sbbd.subs(Acn_bt,q_s*vcn_bt/(c*K*r_mag*gm))

#dcv_aph_Acv_bt_sbbd = dcv_aph_Acv_bt_sbbd.subs(q_s,1)
#dcv_aph_Acv_bt_sbbd = expand(dcv_aph_Acv_bt_sbbd.subs(rcn_bt*vcv_bt,c*K*r_mag*gm))
#dcv_aph_Acv_bt_sbbd = dcv_aph_Acv_bt_sbbd.subs(rcn_bt*vcv_bt,c*K*r_mag*gm)
#dcv_aph_Acv_bt_sbbd = dcv_aph_Acv_bt_sbbd.subs(rcv_aph*rcn_aph,0)
#dcv_aph_Acv_bt_sbbd = dcv_aph_Acv_bt_sbbd.subs(rcn_aph*rcv_aph,0) #OLD SIMPLIFICATION -leaves a Vdot dependence
#dcv_aph_Acv_bt_sbbd = dcv_aph_Acv_bt_sbbd.subs(vcv_aph*vcv_bt,c**2*gup_ab) 

In [25]:
expand(dcv_aph_Acv_bt_sbbd)

\dot{V^\beta}*\tilde{r}^{\alpha}*q_s/(K**2*c**2*gamma**2*|\tilde{r}|**2) - q_s*v^alpha*v^beta/(K**2*c**2*gamma**2*|\tilde{r}|**2) - \dot{V^\alpha}*\tilde{r}^{\alpha}*\tilde{r}_{\alpha}*q_s*v^beta/(K**3*c**3*gamma**3*|\tilde{r}|**3) + \tilde{r}^{\alpha}*q_s*v^beta/(K**3*c*gamma**3*|\tilde{r}|**3)

In [26]:
expand(dcv_aph_Acv_bt_alt)-expand(dcv_aph_Acv_bt_sbbd)

0

In [27]:
rcv_bt*vcn_bt

\tilde{r}^{\beta}*v_beta

### This is our main result, before substituting in explicit components. Note that I use $V_{\beta-observer}$ instead of $P_\beta - (q/c) A_\beta$

In [28]:
Pcn_bt_expl = m_ob*vcn_bt_ob+q*q_s*vcn_bt/(c**2*(K)*r_mag*gm)

In [29]:
Pcn_bt_expl

m_{ob}*v_beta_-ob + q*q_s*v_beta/(K*c**2*gamma*|\tilde{r}|)

In [30]:
#dPcv_aph_dtau_O = q/c*(vcn_bt_ob-q/(m_ob*c)*Acn_bt)*dcv_aph_Acv_bt_sbbd #original def... spurious
dPcv_aph_dtau = q/(c*m_ob)*(Pcn_bt-q/c*Acn_bt)*dcv_aph_Acv_bt_sbbd
#dPcv_aph_dtau = q/(c*m_ob)*(m_ob*vcn_bt_ob)*dcv_aph_Acv_bt_sbbd

dPcv_aph_dtau_sbbd = dPcv_aph_dtau.subs(Acn_bt,q_s*vcn_bt/(c*K*r_mag*gm))
dPcv_aph_dtau_sbbd = dPcv_aph_dtau_sbbd.subs(Pcn_bt,Pcn_bt_expl)
dPcv_aph_dtau_sbbd = expand(dPcv_aph_dtau_sbbd)
dPcv_aph_dtau_sbbd = dPcv_aph_dtau_sbbd.subs(gup_ab*vcn_bt_ob,vcv_aph_ob)

In [66]:
expand(dPcv_aph_dtau_sbbd)

\dot{V^\beta}*\tilde{r}^{\alpha}*q*q_s*v_beta_-ob/(K**2*c**3*gamma**2*|\tilde{r}|**2) - q*q_s*v^alpha*v^beta*v_beta_-ob/(K**2*c**3*gamma**2*|\tilde{r}|**2) - \dot{V^\alpha}*\tilde{r}^{\alpha}*\tilde{r}_{\alpha}*q*q_s*v^beta*v_beta_-ob/(K**3*c**4*gamma**3*|\tilde{r}|**3) + \tilde{r}^{\alpha}*q*q_s*v^beta*v_beta_-ob/(K**3*c**2*gamma**3*|\tilde{r}|**3)

In [32]:
#From Jackson 14.12:
dvdtau_0= c*gm**4*bta_dot_dbta
dvdtau_3= c*gm**2*dbta+c*gm**4*bta*(bta_dot_dbta)

v0 = gm*c
v3 = gm*c*bta_x
v0_ob = gm_ob*c
v3_ob = gm_ob*c*bta_ob
v0_dot = gm**4*bta_dot_dbta*c #see jackson 14.12 or wikipedia "four-acceleration"
v3_dot = gm**2*bta_dot*c+gm**4*bta_dot_dbta*bta*c

r0 = r_mag
r3 = r_mag*n_src_ob

In [33]:
bta_dot

\dot{\beta}

In [34]:
dPcv_aph_dtau_0 = expand(dPcv_aph_dtau_sbbd)

#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcv_aph*vcv_bt,gup_ab*c**2)
#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(rcv_aph*vcv_bt,gup_ab*K*c*r_mag*gm)
#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(gup_ab*vcn_bt_ob,v0)


#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(dvdtau_bt*vcn_bt_ob,v0_ob*v0_dot-v3_ob*v3_dot)
dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(dvdtau_aph,v0_dot)
dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(rcv_aph,r0)
dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(rcn_aph,r0) #


#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(dvdtau_bt,dvdtau_0)
#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(dvdtau_aph,dvdtau_0)

#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(gup_ab*vcn_bt_ob,vcv_aph_ob)
#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcv_bt*vcn_bt_ob,c**2*gm_ob*gm*(1-bta_dot_btaob))
#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcv_bt*vcn_bt_ob,v0*v0_ob-v3*v3_ob)

#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcn_aph,v0)
dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcv_aph,v0)

#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcn_bt,v0)
#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcv_bt,v0)


# #dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(rcn_bt,r0)
# dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(rcv_bt,r0)
# dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(gup_ab*vcn_bt_ob,v0_ob)
#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcv_aph_ob,v0_ob)
#dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(dvdtau_bt,v0_dot)

# #dPcv_aph_dtau_0 = dPcv_aph_dtau_0.subs(vcn_aph_ob,-v0_ob)

In [67]:
dPcv_aph_dtau_0

\dot{V^\beta}*q*q_s*v_beta_-ob/(K**2*c**3*gamma**2*|\tilde{r}|) - q*q_s*v^beta*v_beta_-ob/(K**2*c**2*gamma*|\tilde{r}|**2) - \vec{\beta}\cdot\dot{\vec{\beta}}*gamma*q*q_s*v^beta*v_beta_-ob/(K**3*c**3*|\tilde{r}|) + q*q_s*v^beta*v_beta_-ob/(K**3*c**2*gamma**3*|\tilde{r}|**2)

In [70]:
dPcv_aph_dtau_0.subs(vcv_bt*vcn_bt_ob,c**2*gm_ob*gm*(1-bta_dot_btaob)).subs(bta_dot_dbta,0).subs(bta_dot,0).subs(dvdtau_bt,0)

-\gamma_{ob}*q*q_s*(1 - \vec{\beta}\cdot\beta_{ob})/(K**2*|\tilde{r}|**2) + \gamma_{ob}*q*q_s*(1 - \vec{\beta}\cdot\beta_{ob})/(K**3*gamma**2*|\tilde{r}|**2)

In [36]:
#full scalar
dPdtau0 = dPcv_aph_dtau_0.subs(vcv_bt*vcn_bt_ob,c**2*gm_ob*gm*(1-bta_dot_btaob))
dPdtau0 = (dPdtau0).subs(dvdtau_bt*vcn_bt_ob,v0_ob*v0_dot-v3_ob*v3_dot)

In [37]:
(dPdtau0)

-\gamma_{ob}*q*q_s*(1 - \vec{\beta}\cdot\beta_{ob})/(K**2*|\tilde{r}|**2) + q*q_s*(\gamma_{ob}*\vec{\beta}\cdot\dot{\vec{\beta}}*c**2*gamma**4 - \gamma_{ob}*beta_ob*c*(\dot{\beta}*c*gamma**2 + \vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*c*gamma**4))/(K**2*c**3*gamma**2*|\tilde{r}|) - \gamma_{ob}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2*q*q_s*(1 - \vec{\beta}\cdot\beta_{ob})/(K**3*c*|\tilde{r}|) + \gamma_{ob}*q*q_s*(1 - \vec{\beta}\cdot\beta_{ob})/(K**3*gamma**2*|\tilde{r}|**2)

In [38]:
expand(dPcv_aph_dtau_sbbd)

\dot{V^\beta}*\tilde{r}^{\alpha}*q*q_s*v_beta_-ob/(K**2*c**3*gamma**2*|\tilde{r}|**2) - q*q_s*v^alpha*v^beta*v_beta_-ob/(K**2*c**3*gamma**2*|\tilde{r}|**2) - \dot{V^\alpha}*\tilde{r}^{\alpha}*\tilde{r}_{\alpha}*q*q_s*v^beta*v_beta_-ob/(K**3*c**4*gamma**3*|\tilde{r}|**3) + \tilde{r}^{\alpha}*q*q_s*v^beta*v_beta_-ob/(K**3*c**2*gamma**3*|\tilde{r}|**3)

In [91]:
dPcv_aph_dtau_3 = expand(dPcv_aph_dtau_sbbd)
dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(dvdtau_aph,v3_dot)
dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(rcv_aph,r3)
dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(rcn_aph,-r3) #
dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcv_aph,v3)
#for verification, above subs are for implementation
#dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcv_bt*vcn_bt_ob,c**2*gm_ob*gm*(1-bta_dot_btaob))
#dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(dvdtau_bt*vcn_bt_ob,v0_ob*v0_dot-v3_ob*v3_dot)

In [92]:
expand(simplify(dPcv_aph_dtau_3))

-\beta_x*q*q_s*v^beta*v_beta_-ob/(K**2*c**2*gamma*|\tilde{r}|**2) + \dot{V^\beta}*q*q_s*v_beta_-ob*|\hat{n}|/(K**2*c**3*gamma**2*|\tilde{r}|) + \dot{\beta}*q*q_s*v^beta*v_beta_-ob*|\hat{n}|**2/(K**3*c**3*gamma*|\tilde{r}|) + \vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma*q*q_s*v^beta*v_beta_-ob*|\hat{n}|**2/(K**3*c**3*|\tilde{r}|) + q*q_s*v^beta*v_beta_-ob*|\hat{n}|/(K**3*c**2*gamma**3*|\tilde{r}|**2)

In [90]:
dPcv_aph_dtau_3.subs(bta_dot_dbta,0).subs(bta_dot,0)

-\beta_x*\gamma_{ob}*q*q_s*(1 - \vec{\beta}\cdot\beta_{ob})/(K**2*|\tilde{r}|**2) + \dot{V^\beta}*q*q_s*v_beta_-ob*|\hat{n}|/(K**2*c**3*gamma**2*|\tilde{r}|) + \gamma_{ob}*q*q_s*|\hat{n}|*(1 - \vec{\beta}\cdot\beta_{ob})/(K**3*gamma**2*|\tilde{r}|**2)

In [52]:
v3

\beta_x*c*gamma

In [53]:
# dPcv_aph_dtau_3 = expand(dPcv_aph_dtau_sbbd)
# dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcv_aph*vcv_bt,gup_ab*c**2)
# dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(rcv_aph*vcv_bt,gup_ab*K*gm*r_mag*c)
# dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(gup_ab*vcn_bt_ob,vcv_aph_ob)
# dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(dvdtau_bt*vcn_bt_ob,-v3_dot*v3_ob)

# # #dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(dvdtau_bt,dvdtau_3)
# # #dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(K,(1-bta*n_src_ob))
# # dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcv_aph,v3)
# # #dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcn_bt,-v3)
# # #dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcv_bt,v3)
# # #dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcv_aph*vcv_bt,gm**2*c**2*bta_dot_dbta)
# # #dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(rcn_aph,-r3)
# dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(rcv_aph,r3)
# # #dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(rcn_bt,r3)
# # #dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(rcv_bt,r3)
# dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcv_aph_ob,gm_ob*c*bta_x_ob)
# # dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcn_bt_ob*vcv_bt,-bta_dot_btaob*c**2*gm*gm_ob)
# # dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(dvdtau_bt,v3_dot)
# # dPcv_aph_dtau_3 = dPcv_aph_dtau_3.subs(vcn_bt_ob,-v3_ob)

NameError: name 'aa' is not defined

In [54]:
simplify(dPcv_aph_dtau_3)

\gamma_{ob}*q*q_s*(K*\beta_x*c*gamma**2*(\vec{\beta}\cdot\beta_{ob} - 1) + K*gamma**2*|\hat{n}|*|\tilde{r}|*(\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2 - beta_ob*(\dot{\beta} + \vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2)) - c*|\hat{n}|*(\vec{\beta}\cdot\beta_{ob} - 1) - gamma**2*|\hat{n}|**2*|\tilde{r}|*(\dot{\beta} + \vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2)*(\vec{\beta}\cdot\beta_{ob} - 1))/(K**3*c*gamma**2*|\tilde{r}|**2)

In [55]:
bta_x_ob

\beta_{x-ob}

In [56]:
dPdtaux = dPcv_aph_dtau_3.subs([[rcv_aph,r_mag*n_src_ob_x],[vcv_aph_ob,bta_ob*c*gm_ob],[gup_ab,-1]])
simplify(dPdtaux)

\gamma_{ob}*q*q_s*(K*\beta_x*c*gamma**2*(\vec{\beta}\cdot\beta_{ob} - 1) + K*gamma**2*|\hat{n}|*|\tilde{r}|*(\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2 - beta_ob*(\dot{\beta} + \vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2)) - c*|\hat{n}|*(\vec{\beta}\cdot\beta_{ob} - 1) - gamma**2*|\hat{n}|**2*|\tilde{r}|*(\dot{\beta} + \vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2)*(\vec{\beta}\cdot\beta_{ob} - 1))/(K**3*c*gamma**2*|\tilde{r}|**2)

In [57]:
energy_momentum = -(dPcv_aph_dtau_0)**2 + (dPcv_aph_dtau_3)**2+m_ob**2*c**2
dynamic_mass_squared = solve(energy_momentum,m_ob**2)
dynamic_mass_squared  = expand(dynamic_mass_squared[0])


In [58]:
bob[0]

NameError: name 'bob' is not defined

In [59]:
dynamic_mass_squared

-\beta_x**2*\gamma_{ob}**2*\vec{\beta}\cdot\beta_{ob}**2*q**2*q_s**2/(K**4*c**2*|\tilde{r}|**4) + 2*\beta_x**2*\gamma_{ob}**2*\vec{\beta}\cdot\beta_{ob}*q**2*q_s**2/(K**4*c**2*|\tilde{r}|**4) - \beta_x**2*\gamma_{ob}**2*q**2*q_s**2/(K**4*c**2*|\tilde{r}|**4) + 2*\beta_x*\dot{\beta}*\gamma_{ob}**2*\vec{\beta}\cdot\beta_{ob}*beta_ob*q**2*q_s**2*|\hat{n}|/(K**4*c**3*|\tilde{r}|**3) - 2*\beta_x*\dot{\beta}*\gamma_{ob}**2*beta_ob*q**2*q_s**2*|\hat{n}|/(K**4*c**3*|\tilde{r}|**3) + 2*\beta_x*\gamma_{ob}**2*\vec{\beta}*\vec{\beta}\cdot\beta_{ob}*\vec{\beta}\cdot\dot{\vec{\beta}}*beta_ob*gamma**2*q**2*q_s**2*|\hat{n}|/(K**4*c**3*|\tilde{r}|**3) - 2*\beta_x*\gamma_{ob}**2*\vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*beta_ob*gamma**2*q**2*q_s**2*|\hat{n}|/(K**4*c**3*|\tilde{r}|**3) - 2*\beta_x*\gamma_{ob}**2*\vec{\beta}\cdot\beta_{ob}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2*q**2*q_s**2*|\hat{n}|/(K**4*c**3*|\tilde{r}|**3) + 2*\beta_x*\gamma_{ob}**2*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2*q

In [ ]:
dPcv_aph_dtau_3

\beta_x*\gamma_{ob}*\vec{\beta}\cdot\beta_{ob}*q*q_s/(K**2*|\tilde{r}|**2) - \gamma_{ob}*\vec{\beta}\cdot\beta_{ob}*q*q_s*|\hat{n}|/(K**3*gamma**2*|\tilde{r}|**2)

In [385]:
#headon

In [157]:
#dPdtauxh = dPdtaux.subs([[dbta*c*gm**2,dbta],[c**3*gm**2,c**2],[bta_dot_dbta*c*gm**4,bta_dot_dbta*gm**2]])

In [158]:
#dPdtauxh

In [159]:
#dPdtauxh = ((dPdtauxh.subs([[n_src_ob,1],[q*q_s,1],[gm_ob,1]])))

In [193]:
#dPdtauxh

In [194]:
#dPdtau0h = ((dPdtau0.subs([[n_src_ob,1],[q*q_s*gm_ob,1]])))

In [195]:
#dPdtau0h

In [32]:
Pfull = (dPdtau0h**2-dPdtaux**2)
#Pfull = (dPdtau0h+dPdtauxh)
Pfull_sbdd = Pfull.subs([[bta_ob,0],[bta_dot_dbta,0]])
Pfull_sbdd2 = Pfull.subs([[bta_ob,-bta],[bta_dot_dbta,0]])
#Pfull_sbdd = Pfull.subs(gm**2,1/(1-bta**2))

In [66]:
n_src_ob

|\hat{n}|

In [34]:
simplify(Pfull)

\gamma_{ob}**2*q**2*q_s**2*(-beta_ob**2*(-K*\vec{\beta}**2*c*gamma**2 + K*gamma**2*|\hat{n}|*|\tilde{r}|*(\dot{\vec{\beta}} + \vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**2) + \vec{\beta}*c*|\hat{n}|)**2 + (K*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**4*|\tilde{r}| - K*c*gamma**2 + c)**2)/(K**6*c**2*gamma**4*|\tilde{r}|**4)

In [35]:
Pow_inputE_ratio = simplify((Pfull/(c*dPdtau0h)))

In [36]:
bta_x

\vec{\beta}_x

In [37]:
bta

\vec{\beta}

In [143]:
 simplify(expand(((((Pow_inputE_ratio.subs(r_mag,K/(gm*(1-bta)))).subs(K,gm*r_mag*(1-bta))).subs(gm_ob,gm)).subs(bta_ob,0)).subs(dbta,0)).subs(bta_dot_dbta,0))

q*q_s*(-\vec{\beta}*gamma**3*|\tilde{r}| + gamma**3*|\tilde{r}| - 1)/(c*gamma**4*|\tilde{r}|**5*(\vec{\beta}**3 - 3*\vec{\beta}**2 + 3*\vec{\beta} - 1))

In [166]:
 simplify(expand(((((Pow_inputE_ratio).subs(K,gm*r_mag*(1-bta))).subs(gm_ob,1)).subs(bta_ob,0))))

q*q_s*(\vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**5*|\tilde{r}|**2 - \vec{\beta}*c*gamma**3*|\tilde{r}| - \vec{\beta}\cdot\dot{\vec{\beta}}*gamma**5*|\tilde{r}|**2 + c*gamma**3*|\tilde{r}| - c)/(c**2*gamma**5*|\tilde{r}|**5*(\vec{\beta}**3 - 3*\vec{\beta}**2 + 3*\vec{\beta} - 1))

In [167]:
 simplify(expand(((((Pow_inputE_ratio.subs(r_mag,K/(gm*(1-bta)))).subs(K,gm*r_mag*(1-bta))).subs(gm_ob,1)).subs(bta_ob,0))))

q*q_s*(\vec{\beta}*\vec{\beta}\cdot\dot{\vec{\beta}}*gamma**5*|\tilde{r}|**2 - \vec{\beta}*c*gamma**3*|\tilde{r}| - \vec{\beta}\cdot\dot{\vec{\beta}}*gamma**5*|\tilde{r}|**2 + c*gamma**3*|\tilde{r}| - c)/(c**2*gamma**5*|\tilde{r}|**5*(\vec{\beta}**3 - 3*\vec{\beta}**2 + 3*\vec{\beta} - 1))

In [185]:
Pow_inputE_ratio_LOWBETA_O = simplify((((((Pow_inputE_ratio).subs(K,gm*r_mag*(1-bta))).subs(gm_ob,1).subs(bta_ob,0).subs(bta_dot_dbta,bta*dbta).subs(n_src_ob,-1)))))
r_sol =solve(Pow_inputE_ratio_LOWBETA_O,r_mag)

In [186]:
r_sol[0]

(-sqrt(c)*gamma**2*sqrt(4*\dot{\vec{\beta}}*\vec{\beta}**2 - 4*\dot{\vec{\beta}}*\vec{\beta} + \vec{\beta}**2*c*gamma - 2*\vec{\beta}*c*gamma + c*gamma) + c*gamma**(5/2)*(\vec{\beta} - 1))/(2*\dot{\vec{\beta}}*\vec{\beta}*gamma**(9/2)*(\vec{\beta} - 1))

In [187]:
r_sol[1]

(sqrt(c)*gamma**2*sqrt(4*\dot{\vec{\beta}}*\vec{\beta}**2 - 4*\dot{\vec{\beta}}*\vec{\beta} + \vec{\beta}**2*c*gamma - 2*\vec{\beta}*c*gamma + c*gamma) + c*gamma**(5/2)*(\vec{\beta} - 1))/(2*\dot{\vec{\beta}}*\vec{\beta}*gamma**(9/2)*(\vec{\beta} - 1))

In [125]:
Pow_inputE_ratio_LOWBETA_0_noaccel = ((((((Pow_inputE_ratio).subs(K,gm*r_mag*(1-bta))).subs(gm_ob,1).subs(bta_ob,0).subs(bta_dot_dbta,bta*dbta).subs(n_src_ob,-1).subs(dbta,0)))))
(Pow_inputE_ratio_LOWBETA_0_noaccel.subs(bta**2,1-1/gm**2))

q*q_s*(-c*gamma**3*|\tilde{r}|*(1 - \vec{\beta}) + c)/(c**2*gamma**5*|\tilde{r}|**5*(1 - \vec{\beta})**3)

In [164]:
Pow_inputE_ratio_equal_op_betas = ((((((Pow_inputE_ratio).subs(K,gm*r_mag*(1-bta))).subs(gm_ob,gm).subs(bta_ob**2,bta**2).subs(bta_dot_dbta,bta*dbta).subs(n_src_ob,-1)))))
#solve(Pow_inputE_ratio_equal_op_betas,r_mag)

In [151]:
Pow_inputE_ratio_simple = (((((Pow_inputE_ratio).subs(K,gm*r_mag*(1-bta))).subs(bta_dot_dbta,bta*dbta).subs(n_src_ob,-1).subs(bta,0).subs(bta_ob,0).subs(gm,1).subs(gm_ob,1))))
(expand(Pow_inputE_ratio_simple.subs(bta**2,1-1/gm**2)))

-q*q_s/(c*|\tilde{r}|**4) + q*q_s/(c*|\tilde{r}|**5)

In [ ]:
(Pfull_sbdd)

In [ ]:
Pfull_sbdd

In [ ]:
sqrt(Pfull_sbdd.subs(bta,0))

In [ ]:
Pfull

In [ ]:
dPdtaux_conv= dPdtaux.subs([[bta_dot_dbta,0],[dbta,0]])
(dPdtaux_conv)

In [ ]:
##seems like a kroenecker needed here on the r*\nu terms in the numerators
##which leads to

In [ ]:
#dP3dtau3 = dP3dtau3.subs(rcv_aph*vcn_bt,-r_mag*c*gm*K)
#dP3dtau3 = dP3dtau3.subs(rcn_bt*vcv_aph,-r_mag*c*gm*K)

#middleman steps, maybe a typo
#dP3dtau3 = dP3dtau3.subs(rcv_aph*vcn_bt,rcn_aph*del_ab*vcv_aph)
#dP3dtau3 = dP3dtau3.subs(rcn_aph*vcv_aph*del_ab,-r_mag*c*gm*K)
#dP3dtau3 = dP3dtau3.subs(rcv_aph,rcn_bt*gup_ba)

dP3dtau3

# take it by hand from here, extract metric tensor and simplify RHS of numerator, 